In [1]:
import pandas as pd
from datetime import date, timedelta

In [2]:
file_path = "https://www.dhhs.vic.gov.au/ncov-covid-cases-by-age-group-csv"
df = pd.read_csv(file_path)

In [3]:
df.tail()

,diagnosis_date,agegroup
641546,2022-02-18,80-89
641547,2022-02-18,40-49
641548,2022-02-18,0-9
641549,2022-02-18,30-39
641550,2022-02-18,70-79


In [4]:
df_pivot = pd.pivot_table(df, index = 'diagnosis_date', columns = 'agegroup', 
                         values = 'agegroup', aggfunc = len, fill_value = 0)

In [5]:
df_pivot.tail()

agegroup,0-9,10-19_,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+
diagnosis_date,,,,,,,,,,
2022-02-14,479,410,280,429,336,181,121,57,42,11
2022-02-15,602,589,427,566,410,227,129,91,29,13
2022-02-16,548,556,481,563,427,234,131,72,31,9
2022-02-17,419,475,399,477,462,228,136,77,59,10
2022-02-18,329,352,267,384,355,161,107,69,30,10


In [6]:
# check that the latest date's sum matches the number of cases reported by Vic govt
# the latest row's date is always yesterday

yesterday = date.today() - timedelta(days = 1)

# the dates in the pivot table are strings
# so use isoformat() to convert date into string before looking it up in the table
df_pivot.loc[yesterday.isoformat()].sum()

2064

In [7]:
# take only the 0-9 and 10-19 cases, from 5 Dec onwards
# for each date, store the previous date's cases; this matches what I've done for NSW
# eg. on 8 Dec, the Vic cases are those reported on the 7th

first = date(2021, 12, 5)
today = date.today()

df_0_19 = pd.DataFrame(columns = ['date', 'cases_0_9', 'cases_10_19'])

curr = first
prev = first - timedelta(days = 1)

while (curr < today):
    
        df_0_19 = df_0_19.append({'date': curr.isoformat(), 'cases_0_9': df_pivot.loc[prev.isoformat(), '0-9'], 
                                  'cases_10_19': df_pivot.loc[prev.isoformat(), '10-19_']}, ignore_index = True)
        prev = curr
        curr = curr + timedelta(days = 1)

In [8]:
df_0_19.tail()

,date,cases_0_9,cases_10_19
71,2022-02-14,447,406
72,2022-02-15,479,410
73,2022-02-16,602,589
74,2022-02-17,548,556
75,2022-02-18,419,475


In [9]:
df_0_19.to_csv("../data/cases_0_19.csv")